In [26]:
import sqlite3
connection = sqlite3.connect('sqlite.db')
cursor = connection.cursor()

Создаём таблицу для данных

In [14]:
query = '''
    CREATE TABLE quotes(
        quote TEXT,
        author TEXT,
        tags TEXT
    )
'''
cursor.execute(query)
connection.commit()

Собираем данные со всех страниц в массив

In [23]:
url = 'https://quotes.toscrape.com/'
new_quotes = []
while True:
    req = requests.get(url)
    soup = bs(req.content, 'html.parser')
    all_quotes = soup.select('div[class=quote]')
    for quote in all_quotes:
        text = quote.find('span', class_='text').text
        author = quote.find('small', attrs={'class': 'author'}).text
        tags = quote.find('meta', class_='keywords')['content']
        new_quotes += [(text, author,tags)]
    try:
        next_page = soup.select_one('li[class=next] a')['href']
        url = 'https://quotes.toscrape.com/' + next_page
        continue
    except:
        break
    else:
         print(f'Ошибка: {req.status_code}')
print('DONE')

DONE


Записываем данные в таблицу

In [28]:
insert_query = '''
    INSERT INTO quotes(
        quote,
        author,
        tags
        
    )
    VALUES(
        ?,
        ?,
        ?
    )
'''
cursor.executemany(insert_query, new_quotes)
connection.commit()

Проверка заполнена ли таблица

In [30]:
list(cursor.execute('SELECT * FROM quotes'))

Удаляем таблицу

In [31]:
cursor.execute('DROP TABLE if exists quotes')

In [34]:
connection.close()